# <center>Colinearity in Regression Model (OLS)

When we are building a regression model, we obviously want to model the relationship between a dependent variable and one or more independent variables.
However, more often than not, we might encounter a situation where the coefficient estimate of each independent variable “doesn’t make sense” and we can’t explain why it occurs. If you ever encounter this situation, there might be collinearity in your regression model.

# What is Collinearity?

- Collinearity occurs because independent variables that we use to build a regression model are correlated with each other. This is problematic because as the name suggests, an independent variable should be independent. It shouldn’t have any correlation with other independent variables.
- If collinearity exists between independent variables, one of the key points of regression analysis is violated. In regression analysis, we want to isolate the influence of each independent variable on our dependent variable. This way, we can interpret the coefficient estimate as the mean change in the dependent variable for each 1 unit change in an independent variable while keeping the other independent variables constant.
- Now if we have collinearity, the key point above is no longer valid, as if we change the value of one independent variable, the other independent variables that are correlated will also change.
- In this post, we are going to see why collinearity becomes such a problem for our regression model, how we can detect it, how it affects our model, and what we can do to remove collinearity.


# Cars DataSet

- To make it more clear why collinearity is such a problem, let’s take a look at a use case.
- For the use case, I am going to use the car dataset that i downloaded it easily on Kaggle.
- Let’s imagine we want to predict the price of a car, or price variable in the dataset. To predict it, we will use certain independent variables such as the car’s city MPG, highway MPG, horsepower, engine size, stroke, width, peak RPM, and compression ratio. Next, we build a regression model based on these independent variables.

In [ ]:
import pandas as pd 
import numpy as np 
import statsmodels.api as sm 

In [ ]:
# data url : 
url = "https://github.com/mostafa-alaywan/Python/raw/main/Data/CarPrice_Assignment.csv"

In [ ]:
cars = pd.read_csv(url)

# EDA

In [ ]:
cars.head()

In [ ]:
cars.shape

In [ ]:
cars.info()

In [ ]:
cars.describe()

In [ ]:
cars.columns

# Missing Value 

In [ ]:
# checking the empty cells or NAN in each variable : no empty cells in cars data set 
cars.isnull().sum()

# Histogram of Price 

In [ ]:
from plotnine import ggplot , aes , geom_histogram , theme_minimal 
ggplot(cars) + aes(x = 'price') + geom_histogram(bins = 20 , fill = "green" , alpha =0.7) + theme_minimal()

# Shapiro's Test

**The Shapiro-Wilk test tests the null hypothesis that the data was drawn from a normal distribution.**

In [ ]:
from scipy import stats                               
shapiro_test = stats.shapiro(cars['price'])
shapiro_test

In [ ]:
cars.columns

# Splitting Data 

In [ ]:
# Features Variables : 
X = cars[['highwaympg','citympg','peakrpm','horsepower','compressionratio','stroke','boreratio','enginesize','carwidth']]

# Target Variable : 
Y = cars['price']

In [ ]:
# We need to add the column of ones to the X to calculate the intercept 𝑏₀.
# statsmodels doesn’t takes 𝑏₀ into account by default. This is just one function call:
X = sm.add_constant(X)
X

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test , Y_train , Y_test = train_test_split( X , Y , test_size = 0.3 , random_state = 1 )

In [ ]:
type(X_train)

# Training The Model 

In [ ]:
import statsmodels.api as sm 
model = sm.OLS(Y_train , X_train).fit()

In [ ]:
model.summary()

# Results Interpretation 

- **Our model is actually doing pretty well, as it has an R² of 85.2%.**
- **Now the problem comes when we try to interpret the model. If we take a look at the coefficient estimate values, the coefficient of highway MPG (135.6009) and city MPG (- 293.0258) have the opposite signs. This doesn’t make sense at all because if a highway MPG of a car increases its price, then the city MPG should also do the same. But this is not the case.**
- **Also, the p-Values of both highway MPG and city MPG are shown insignificant by the model (>0.05), indicating that we can exclude them from our regression model. But are they actually insignificant?**
- **If we reckon that there might be a slight chance of collinearity in the model, we shouldn’t trust this p-Value straight away.**

# Detecting Collinearity

**There are two easy ways to detect if collinearity exists in our regression model.The first one is by looking at the pairplot or the correlation matrix of our independent variables. The rule of thumb is that if two independent variables have a Pearson’s correlation above 0.9, then we can say that both independent variables are highly correlated with each other and thus, they are collinear.**

### Pairplot

In [ ]:
# pairs plot :
import seaborn as sn
sn.pairplot(X)

### Correlation Matrix

In [ ]:
CorrMatrix = X.corr()
CorrMatrix

In [ ]:
# annot = True : show pearson correlation coefficient 
sn.heatmap(CorrMatrix ,annot =True)

In [ ]:
# set_precision(2) : two digits 
CorrMatrix.style.background_gradient(cmap='coolwarm').set_precision(2)

**From the image above, we can clearly see that highway MPG and city MPG are highly correlated, as they have a Pearson’s correlation of 0.97. As they have a positive correlation, this means that if we increase the highway MPG, the city MPG will also increase by almost the same amount.**

### Variance Inflation Factor

**Variance Inflation Factor or VIF measures the influence of collinearity on the variance of our coefficient estimates. VIF can be described mathematically as follows:**

In [ ]:
from IPython import display
display.Image("C:/Users/Mostafa/Desktop/Python/VIF.png")

- **From the equation above, we know that if R² of independent variable xi is large or close to 1, then the corresponding VIF of xi would be large as well. This means that independent variable xi can be explained by other independent variables. In other words, xi is highly correlated with other independent variables. Thus, the variance of the coefficient estimate βi is also high.**
- **We can use statsmodel library to calculate VIF in Python as follows**

In [ ]:
#calculate VIF for each explanatory variable
# X.shape[1] = number of column in X 
# X.values : transform X from DataFrame to ndarray

from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['variable'] = X.columns

#view VIF for each explanatory variable 
vif

In [ ]:
# transpose the Data Frame vif : 
vif.T

In [ ]:
type(vif)

### How to Interpret VIF Values

- **The value for VIF starts at 1 and has no upper limit. A general rule of thumb for interpreting VIFs is as follows:**

- **A value of 1 indicates there is no correlation between a given explanatory variable and any other explanatory variables in the model.**
- **A value between 1 and 5 indicates moderate correlation between a given explanatory variable and other explanatory variables in the model, but this is often not severe enough to require attention.**
- **A value greater than 5 indicates potentially severe correlation between a given explanatory variable and other explanatory variables in the model. In this case, the coefficient estimates and p-values in the regression output are likely unreliable.**
- **In our use case, we can see that highway MPG and city MPG have VIF scores way above 10, indicating that they are highly correlated with each other. We can see this phenomenon as well from the correlation matrix above.**


# Removing Collinearity

**Now that we know severe collinearity exists in our independent variables, we need to find a way to fix this. There are two common ways to remove collinearity.**

### Principal Component Regression  and Variable Selection

- **The variable selection the most straightforward solution to remove collinearity and oftentimes, domain knowledge would be extremely helpful to achieve the best solution. To remove collinearity, we can exclude independent variables that have a high VIF value from our regression model.**
- **We know that highway MPG and city MPG have high VIF values. If we have domain knowledge, we know that it’s not necessary to exclude both from our regression model. Instead, we only need to pick either one of them. Let’s say that we exclude highway MPG from our model.**
- **We also know that both horsepower and engine size have moderately high VIF values as well. Although they measure different things, higher engine size generally corresponds to the higher horsepower. Thus, we exclude horsepower from our model.**
- **In the end, we built our regression model once again, but this time without highway MPG and horsepower.**

In [ ]:
X_train.columns

In [ ]:
X_selection = X_train[['const', 'citympg', 'peakrpm','compressionratio', 'stroke', 'boreratio', 'enginesize', 'carwidth']]

Y_selection = Y_train

In [ ]:
model_selection = sm.OLS(Y_selection , X_selection) .fit()
model_selection.summary()

In [ ]:
#calculate VIF for each explanatory variable

from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_sel = pd.DataFrame()
vif_sel['VIF'] = [variance_inflation_factor(X_selection.values, i) for i in range(X_selection.shape[1])]
vif_sel['variable'] = X_selection.columns

#view VIF for each explanatory variable 
vif_sel

**And now we don’t have severe collinearity among independent variables anymore! Now we can proceed with our regression analysis.**

# The Effect of Removing Collinearity in Regression Model

- **Now that we remove collinearity in our independent variables, let’s compare the regression model with collinearity and without collinearity.**
- **The problem with collinearity is that it will inflate the variance or standard error of coefficient estimates.**
- **If we take a look at both models, the standard error of coefficient estimates in the regression model without collinearity is much less compared to the model with collinearity. The most apparent one is the city MPG variable. When collinearity exists, the standard error of the coefficient estimates of this variable is 169.14 compared to 60.30 when collinearity is removed.**
- **If we take a look at the p-Value, the model with collinearity concludes that the city MPG variable is statistically insignificant, meaning we can exclude this variable from our model.**
- **However, when we remove the collinearity, the p-Value of this variable is 0.003, which is actually statistically significant. Collinearity will inflate the variance of coefficient estimates of independent variables, making it difficult for us to trust the resulting p-Value from the model.**
- **Moreover, with sufficient variable selection, the F-statistics of the model without collinearity is much more significant than the model with collinearity, although the model with collinearity has more independent variables.**

# Do We Need to Remove Collinearity?

- **One important thing that we should notice is that collinearity would not affect the model prediction or the accuracy of the model. If you look at the R² comparison between the model with and without collinearity above, they are both similar. In fact, the model with collinearity typically yields better accuracy due to the fact it generally has more independent variables.**
- **Collinearity only affects the variance of coefficient estimates and the p-Values.**
- **So, if we want to build a regression model to make predictions and we don’t need to understand the influence of each independent variable, then we don’t need to remove collinearity in our model.**
- **However, if model interpretation is important for us and we need to understand the influence of each independent variable on the model’s prediction, then removing collinearity in our model is necessary.**

<center>-----------------------------------------------------The End--------------------------------------------------------------------